In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from tqdm import tqdm
from collections import defaultdict
from math import radians, sin, cos, atan2, sqrt

def calculate_bearing(p1, p2):
    lat1, lon1 = radians(p1[0]), radians(p1[1])
    lat2, lon2 = radians(p2[0]), radians(p2[1])
    dlon = lon2 - lon1

    x = sin(dlon) * cos(lat2)
    y = cos(lat1)*sin(lat2) - sin(lat1)*cos(lat2)*cos(dlon)
    bearing = atan2(x, y)
    return bearing

parquet_file = pq.ParquetFile("../../Data_Preperation/filtered_data_full_1.parquet")
num_row_groups = parquet_file.num_row_groups
print(f"Processing {num_row_groups} row groups")

id_tracking_data = defaultdict(list)

for rg in tqdm(range(num_row_groups), desc="Reading row groups"):
    df = parquet_file.read_row_group(rg).to_pandas()
    for tracking_id, group in df.groupby("id_tracking"):
        id_tracking_data[tracking_id].append(group)

features = []

for tracking_id, parts in tqdm(id_tracking_data.items(), desc="Extracting features"):
    group = pd.concat(parts).sort_values(by="sequence").dropna(subset=["latitude", "longitude"])

    coords = list(zip(group["latitude"], group["longitude"]))
    if len(coords) < 2:
        continue

    num_points = len(coords)
    
    lat_min, lat_max = group["latitude"].min(), group["latitude"].max()
    lon_min, lon_max = group["longitude"].min(), group["longitude"].max()

    height = geodesic((lat_min, lon_min), (lat_max, lon_min)).meters
    width = geodesic((lat_min, lon_min), (lat_min, lon_max)).meters
    bbox_area = height * width
    
    point_density = num_points / (bbox_area + 1e-6)
    dists = [geodesic(coords[i], coords[i+1]).meters for i in range(len(coords)-1)]
    avg_segment_distance = np.mean(dists)
    total_distance = np.sum(dists)

    num_stops = (group["speed"] == 0).sum()
    duration = max(group["tracking_duration"]) if "tracking_duration" in group.columns else 0
    length = max(group["tracking_length"]) if "tracking_length" in group.columns else 0
    
    straight_line = geodesic(coords[0], coords[-1]).meters
    straightness = straight_line / total_distance if total_distance > 0 else 0

    angle_changes = []
    for i in range(1, len(coords) - 1):
        prev_bearing = calculate_bearing(coords[i - 1], coords[i])
        next_bearing = calculate_bearing(coords[i], coords[i + 1])
        delta = abs(next_bearing - prev_bearing)
        if delta > np.pi:
            delta = 2 * np.pi - delta
        angle_changes.append(delta)
    mean_heading_change = np.mean(angle_changes) if angle_changes else 0

    features.append({
        "tracking_id": tracking_id,
        "num_points": num_points,
        "bbox_area": bbox_area,
        "point_density": point_density,
        "avg_segment_distance": avg_segment_distance,
        "total_distance": total_distance,
        "straightness": straightness,
        "mean_heading_change": mean_heading_change,
        "num_stops": num_stops,
        "duration": duration,
        "length": length
    })

features_df = pd.DataFrame(features)
features_df.to_csv("tracking_features_chunked_2_advanced_featurs.csv", index=False)
print("Feature extraction complete. Saved to tracking_features_chunked.csv")


Processing 65 row groups


Extracting features: 100%|██████████| 45598/45598 [1:58:40<00:00,  6.40it/s]  


Feature extraction complete. Saved to tracking_features_chunked.csv


: 